In [208]:
from dataclasses import dataclass

import pickle

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ChromeOptions

from bs4 import BeautifulSoup

In [5]:
options = ChromeOptions()
options.add_argument("--headless=new")
driver = webdriver.Chrome(options=options)

In [7]:
url = 'https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=0&it_search_input=a&companyid=&sectorid='
driver.get(url)

In [31]:
soup = BeautifulSoup(driver.page_source, 'html')

In [49]:
table = soup.find('table')
table

<table class="stockTable" summary="Investment trust search results"><thead><tr><th>Identifier</th><th>Title</th><th colspan="2">Description</th></tr></thead><tbody>
<tr><td colspan="4"><table>
<tbody><tr>
<td style="border:0pt none"> </td>
<td style="border:0pt none">
Page: 
<strong>1</strong>
<a href="https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=50&amp;it_search_input=a&amp;companyid=&amp;sectorid=" title="View page 2">2</a>
<a href="https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=100&amp;it_search_input=a&amp;companyid=&amp;sectorid=" title="View page 3">3</a>
<a href="https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=150&amp;it_search_input=a&amp;companyid=&amp;sectorid=" title="View page 4">4</a>
<a href="https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=200&amp;it_search_input=a&amp;companyid=&amp;sectorid=" title="View page 5">5</a>
<a href="https

In [123]:
rows = table.find_all('tr')

In [149]:
# display(rows[0])
# display(rows[1])
# display(rows[3])

td_rows = rows[3].findAll('td')
td_rows[0].findAll('td')

[]

In [53]:
# rows = rows[3:]
# rows
# rows[0]

[<tr>
 <td>ASIT</td>
 <td><a class="link-headline" href="https://www.hl.co.uk/shares/shares-search-results/BYPBD39" title="View factsheet for Aberforth Split Level Income Trust plc Ord GBP0.01">Aberforth Split Level Income Trust plc</a></td>
 <td>Ord GBP0.01</td>
 <td class="-dataCell">
 <a class="button" href="https://online.hl.co.uk/my-accounts/security_deal/sedol/BYPBD39" title="Place deal for Aberforth Split Level Income Trust plc Ord GBP0.01 now">Deal</a>
 </td>
 </tr>,
 <tr>
 <td>AAS</td>
 <td><a class="link-headline" href="https://www.hl.co.uk/shares/shares-search-results/BMF19B5" title="View factsheet for Abrdn Asia Focus Plc ORD GBP0.05">Abrdn Asia Focus Plc</a></td>
 <td>ORD GBP0.05</td>
 <td class="-dataCell">
 <a class="button" href="https://online.hl.co.uk/my-accounts/security_deal/sedol/BMF19B5" title="Place deal for Abrdn Asia Focus Plc ORD GBP0.05 now">Deal</a>
 </td>
 </tr>,
 <tr>
 <td>AAIF</td>
 <td><a class="link-headline" href="https://www.hl.co.uk/shares/shares-sea

In [148]:
def get_symbol_row_data(symbol_row):
    symbol_rows = symbol_row.findAll('td')
    symbol = symbol_rows[0].text
    name = symbol_rows[1].text
    link = symbol_rows[1].find('a', href=True)['href']
    dealable = symbol_rows[3].text.startswith('\nDeal')

    return symbol, name, link, dealable

In [161]:
def check_symbol_row(symbol_row):
    td_rows = symbol_row.findAll('td')
    
    if len(td_rows) != 4:
        return False
    
    td_row_1 = td_rows[0].findAll('td')

    return len(td_row_1) == 0

    # td_rows = symbol_row[0].findAll('td')
    # return len(td_rows.findAll('td')) == 0

check_symbol_row(rows[3])

True

In [209]:
@dataclass
class InvestmentTrust:
    symbol: str
    name: str
    link: str
    tradeable: bool

In [213]:
investment_trusts = []

for symbol_row in rows:
    if not check_symbol_row(symbol_row):
        continue
    symbol, name, link, dealable = get_symbol_row_data(symbol_row)
    # print(symbol, name, link, dealable)
    investment_trusts.append(InvestmentTrust(symbol, name, link, dealable))

[InvestmentTrust(symbol='III', name='3i Group Plc', link='https://www.hl.co.uk/shares/shares-search-results/B1YW440', tradeable=True),
 InvestmentTrust(symbol='3IN', name='3i Infrastructure plc', link='https://www.hl.co.uk/shares/shares-search-results/BF5FX16', tradeable=True),
 InvestmentTrust(symbol='ASL', name='Aberforth Smaller Companies Trust plc', link='https://www.hl.co.uk/shares/shares-search-results/0006655', tradeable=True),
 InvestmentTrust(symbol='ASIT', name='Aberforth Split Level Income Trust plc', link='https://www.hl.co.uk/shares/shares-search-results/BYPBD39', tradeable=True),
 InvestmentTrust(symbol='AAS', name='Abrdn Asia Focus Plc', link='https://www.hl.co.uk/shares/shares-search-results/BMF19B5', tradeable=True),
 InvestmentTrust(symbol='AAIF', name='Abrdn Asian Income Fund Limited', link='https://www.hl.co.uk/shares/shares-search-results/B0P6J83', tradeable=True),
 InvestmentTrust(symbol='ACIC', name='Abrdn China Investment Company Limited', link='https://www.hl.c

In [197]:
num_pages = 0

for link in soup.findAll('a', href=True):
    # print(link.title)
    title = link.get('title')
    # print(title)

    if not title:
        continue

    if title.startswith('View page '):    
        print(title)
        num_pages += 1
    
num_pages = num_pages // 2
max_offset = num_pages * 50
step_size = 50
print(max_offset)

View page 2
View page 3
View page 4
View page 5
View page 6
View page 7
View page 8
View page 2
View page 3
View page 4
View page 5
View page 6
View page 7
View page 8
350


In [201]:
i = 0
while i <= max_offset:
    url = f'https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset={i}&it_search_input=a&companyid=&sectorid='
    print(url)
    i += step_size

# for i in range(0, max_offset, 50):
#     # print(i)
#     url = f'https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset={i}&it_search_input=a&companyid=&sectorid='
#     print(url)

https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=0&it_search_input=a&companyid=&sectorid=
https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=50&it_search_input=a&companyid=&sectorid=
https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=100&it_search_input=a&companyid=&sectorid=
https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=150&it_search_input=a&companyid=&sectorid=
https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=200&it_search_input=a&companyid=&sectorid=
https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=250&it_search_input=a&companyid=&sectorid=
https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=300&it_search_input=a&companyid=&sectorid=
https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=350&it_search_input=a&companyid=&sectorid=


In [206]:
def get_page_urls():
    num_pages = 0

    for link in soup.findAll('a', href=True):
        title = link.get('title')

        if not title:
            continue

        if title.startswith('View page '):
            num_pages += 1
        
    num_pages = num_pages // 2
    step_size = 50
    max_offset = num_pages * step_size

    page_urls = []
    i = step_size
    while i <= max_offset:
        url = f'https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset={i}&it_search_input=a&companyid=&sectorid='
        page_urls.append(url)
        i += step_size
    
    return page_urls

page_urls = get_page_urls()
print(page_urls)

['https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=50&it_search_input=a&companyid=&sectorid=', 'https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=100&it_search_input=a&companyid=&sectorid=', 'https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=150&it_search_input=a&companyid=&sectorid=', 'https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=200&it_search_input=a&companyid=&sectorid=', 'https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=250&it_search_input=a&companyid=&sectorid=', 'https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=300&it_search_input=a&companyid=&sectorid=', 'https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=350&it_search_input=a&companyid=&sectorid=']


In [215]:
for page_url in page_urls:
    print(page_url)
    driver.get(page_url)
    soup = BeautifulSoup(driver.page_source, 'html')
    table = soup.find('table')
    rows = table.find_all('tr')

    for symbol_row in rows:
        if not check_symbol_row(symbol_row):
            continue
        symbol, name, link, dealable = get_symbol_row_data(symbol_row)
        # print(symbol, name, link, dealable)
        investment_trusts.append(InvestmentTrust(symbol, name, link, dealable))


https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=50&it_search_input=a&companyid=&sectorid=
https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=100&it_search_input=a&companyid=&sectorid=
https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=150&it_search_input=a&companyid=&sectorid=
https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=200&it_search_input=a&companyid=&sectorid=
https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=250&it_search_input=a&companyid=&sectorid=
https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=300&it_search_input=a&companyid=&sectorid=
https://www.hl.co.uk/shares/investment-trusts/search-for-investment-trusts?offset=350&it_search_input=a&companyid=&sectorid=


In [223]:
inv_trusts_filename = 'investment_trusts_231002.pkl'
with open(inv_trusts_filename, "wb") as handle:
    pickle.dump(investment_trusts, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [224]:
with open(inv_trusts_filename, "rb") as handle:
    it2 = pickle.load(handle)

it2

[InvestmentTrust(symbol='III', name='3i Group Plc', link='https://www.hl.co.uk/shares/shares-search-results/B1YW440', tradeable=True),
 InvestmentTrust(symbol='3IN', name='3i Infrastructure plc', link='https://www.hl.co.uk/shares/shares-search-results/BF5FX16', tradeable=True),
 InvestmentTrust(symbol='ASL', name='Aberforth Smaller Companies Trust plc', link='https://www.hl.co.uk/shares/shares-search-results/0006655', tradeable=True),
 InvestmentTrust(symbol='ASIT', name='Aberforth Split Level Income Trust plc', link='https://www.hl.co.uk/shares/shares-search-results/BYPBD39', tradeable=True),
 InvestmentTrust(symbol='AAS', name='Abrdn Asia Focus Plc', link='https://www.hl.co.uk/shares/shares-search-results/BMF19B5', tradeable=True),
 InvestmentTrust(symbol='AAIF', name='Abrdn Asian Income Fund Limited', link='https://www.hl.co.uk/shares/shares-search-results/B0P6J83', tradeable=True),
 InvestmentTrust(symbol='ACIC', name='Abrdn China Investment Company Limited', link='https://www.hl.c

In [227]:
for it in it2:
    if it.symbol == 'JEMA':
        print(it)

InvestmentTrust(symbol='JEMA', name='JPMorgan Emerging Europe, Middle East & Africa Sec', link='https://www.hl.co.uk/shares/shares-search-results/3216473', tradeable=True)
